# Lab 1

## Init

![Task](image.png)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tabulate as tb

Data = np.array([
    19.8, 19.1, 19.3, 18.8, 20.2, 20.8, 20.7, 19.7, 19.6, 19.2,
    20.9, 20.9, 20.2, 19.6, 20.4, 20.4, 20.2, 20.4, 18.9, 19.7,
    19.8, 20.6, 20.7, 19.7, 20.3, 19.8, 20.4, 20.3, 20.6, 20.5,
    20.4, 20.5, 20.3, 20.5, 20.2, 20.5, 20.7, 21.0, 20.4, 20.8,
    20.5, 20.4, 20.6, 21.0, 20.4, 20.4, 20.3, 19.7, 19.9, 20.1
], type(float))

lenData = len(Data)
# print(tb.tabulate(np.array_split(Data, 10)))


## Frequencies (1)

### Discrete series (1.1)

In [ ]:
sortedData = np.sort(Data)
orderedData, frequencies, = np.unique(sortedData, return_counts=True)
lenOrderedData = len(orderedData)
barColumnSize = (orderedData[-1] - orderedData[0]) / lenOrderedData

In [ ]:
print(tb.tabulate([orderedData, frequencies], showindex=['Xi', 'Frequency']))

bar = plt.bar(orderedData, frequencies,
              color='purple', edgecolor='k',
              width=barColumnSize)
plt.title('Discrete series')
plt.xlabel('Value')
plt.ylabel('Frequency')

### Intervals series (1.2)

In [ ]:
R = sortedData[-1] - sortedData[0]
k = np.sqrt(lenData)
h = np.round(R / k, 1)
offset = 0.5 * h
x0 = sortedData[0] - offset
lenSeries = int(np.round(k, 1))
sessions = np.array([(.0, .0)] * lenSeries)
sessionElementsCount = np.array([1] * lenSeries)

for i in range(lenSeries):
    sessionEnd = x0 + h
    sessions[i] = (x0, sessionEnd)
    sessionElementsCount[i] = \
        np.sum(((sortedData >= x0) & (sortedData < sessionEnd)))
    x0 += h

In [ ]:
print(
    tb.tabulate(
        [[f'{session[0]:.2f} - {session[1]:.2f}' for session in sessions],
         sessionElementsCount]))

positions = [session[0] + offset for session in sessions]
plt.bar(positions, sessionElementsCount,
        color='purple', edgecolor='k',
        width=h)
plt.plot(positions, sessionElementsCount,
         color="k", linewidth=2)
plt.title('Interval series')
plt.xlabel('Interals')
plt.ylabel('Frequency')


## Sum curve (2-3)

In [ ]:
relatedFrequencies = np.array([np.round(fr / lenData, 2) for fr in frequencies])
accumulatedRelativeFrequencies = [.0] * lenOrderedData
accumulatedRelativeFrequencies[0] = relatedFrequencies[0] 
for i in range(1, lenOrderedData):
    accumulatedRelativeFrequencies[i] = \
        accumulatedRelativeFrequencies[i - 1] + relatedFrequencies[i]

In [ ]:
print(tb.tabulate(
    [orderedData,
     relatedFrequencies,
     accumulatedRelativeFrequencies]))

plt.bar(orderedData, accumulatedRelativeFrequencies,
        width=(orderedData[-1] - orderedData[0]) / lenOrderedData, color='purple', edgecolor='k')
plt.plot(orderedData, accumulatedRelativeFrequencies,
        linewidth=2, color='k')
plt.title('Sum curve')
plt.xlabel('Interal center')
plt.ylabel('Frequency')

## Task (4)

In [ ]:
MoX, = orderedData[np.where(frequencies == frequencies.max())]
MeX = .0
half = lenOrderedData / 2
if (lenOrderedData % 2 == 0):
    MeX = np.round((orderedData[half] + orderedData[half + 1]) / 2, 2)
else:
    MeX = orderedData[(int(np.round(half, 0)))]

u = np.array((orderedData - MoX) / h)


In [ ]:
print('Самая частовстречаемая пропускная сп-ть скважин: ', MoX)
print('Пропускная сп-ть скважин, делящая данные выборки на равные части: ', MeX)

In [ ]:
x_0 = np.array([
    [orderedData[i], frequencies[i], u[i],
     frequencies[i] * u[i],
     frequencies[i] * u[i]**2,
     frequencies[i] * u[i]**3,
     frequencies[i] * u[i]**4]
    for i in range(lenOrderedData)])

m1, m2, m3, m4 = 0, 0, 0, 0

for i in range(lenOrderedData):
    m1 += x_0[i, 3]
    m2 += x_0[i, 4]
    m3 += x_0[i, 5]
    m4 += x_0[i, 6]

m1 /= lenData
m2 /= lenData
m3 /= lenData
m4 /= lenData


In [ ]:
headers = [
    'Xi', 'Ni', 'Ui', 'Ni * Ui',
     'Ni * Ui^2', 'Ni * Ui^3', 'Ni * Ui^4']

print(tb.tabulate(x_0, headers=headers, floatfmt='.2f'))

In [ ]:
print(f'M1: {m1:.2f}')
print(f'M2: {m2:.2f}')
print(f'M3: {m3:.2f}')
print(f'M4: {m4:.2f}')
s_2 = (m2 - m1**2) * h**2
s = np.sqrt(s_2)
cA = m1 * h + MoX
print(f'Chosen avarage (X̄) (Средняя пропускная способность скважин): {cA:.2f}')
print(f'S (Степень рассеяния пропускной способности скважин относительно X̄): {s:.2f}')
print(f'V (Колеблеммость значений пр-ой спос-ти скважин): {s / cA:.2f}')
m_3 = (m3 - 3 * m2 * m1 + 2 * m1**3) * h**3
m_4 = (m4 - 4 * m3 * m1 + 6 * m2 * m1**2 - 3 * m1**4) * h**4
As = m_3 / s**3
Ex = m_4 / s**4 - 3
print(f'As (Отклонения значений пр-ой сп-ти скважин от норм-го распределения): {As:.2f}')
print(f'Ex (Отклонения значений пр-ой сп-ти скважин от норм-го распределения): {Ex:.2f}')


## Task (5)

In [ ]:
ty = 2.009
q = 0.21
print(f'{cA - s / lenOrderedData * ty:.2f} < a < {cA + s / lenOrderedData * ty:.2f}')
print('=> Средняя пропускная способность скважин находится в пределах a')
print(f'{s * (1 - q):.2f} < σ < {s * (1 + q):.2f}')
print('=> Отклонения истинных значений пр-ой сп-ти скважин не выходят за промежутки σ')



# Lab 2

## Empirical and theoretical curves (1)

In [ ]:
import math
import scipy as sc

x_0 = [0] * lenOrderedData
theoricalFrequencies = [0] * lenOrderedData

for i in range(lenOrderedData):
    q1 = orderedData[i] - cA
    q2 = q1 / s
    q3 = sc.stats.norm.cdf(q2) - 0.5
    q4 = orderedData[i] * h / s * q3
    q5 = np.round(q4, 0)
    x_0[i] = [orderedData[i], frequencies[i],
              q1, q2, q3, q4, q5]
    theoricalFrequencies[i] = q5


In [ ]:
headers = [
    'Xi', 'Ni', 'Xi - X̄', 'Ui = (Xi - X̄) / S',
    'φ(Ui)', 'Yi = nh / S * φ(Ui)', 'N`']

print(tb.tabulate(x_0, headers=headers, floatfmt='.2f',
      tablefmt="github", numalign='center'))


In [ ]:
plt.plot(orderedData, theoricalFrequencies,
         color='darkblue', label='Theorical curve', linestyle='--')
plt.plot(orderedData, frequencies, color='darkgreen', label='Emperical curve')
plt.xlabel('Values Xi')
plt.ylabel('Frequency')
plt.legend()


## Checking/Testing

### Pearson test

In [ ]:
x_0 = 0
k = 4

for i in range(lenOrderedData):
    f = frequencies[i]
    tf = theoricalFrequencies[i]
    q1 = f - tf
    q2 = q1**2
    if tf != 0:
        q3 = q2 / tf
    else:
        q3 = 0
    x_0 += q3

# In приложение
a = 13.277

if (x_0 < a):
    check = ''
else:
    check = ' не'

print(
    f"Данные выборки, харак-щие пр-ую сп-ть скважин,{check} подчиняются нормальному закону распределения")


### Romanovskiy test

In [ ]:
if (np.abs((x_0**2 - k) / np.sqrt(2 * k)) < 3):
    check = ' не'
    check2 = ''
else:
    check = ''
    check2 = ' не'

print(
    f"Расхождение между эмпирическим и теоритическим распределением{check} существенны\n=>{check2} подчиняются нормальному закону распределения")


### Approximate test

In [ ]:
n = lenData
SAs = np.sqrt((6 * (n - 1)) / ((n + 1) * (n + 3)))
SEx = np.sqrt((24 * n * (n - 2) * (n - 3)) / ((n - 1)**2 * (n + 3) * (n + 5)))

if (np.abs(As) > SAs and np.abs(Ex) < SEx):
    check = ' не'
else:
    check = ''

print(
    f"Данные выборки, харак-щие пр-ую сп-ть скважин,{check} подчиняются нормальному закону распределения")
